# Tutorial 1 - Getting Started

This is a tutorial for Brightway2, an open source framework for Life Cycle Assessment. This tutorial will cover the basics of databases and activities and looking at LCI databases and LCIA methods.

You will get the most from this tutorial as part of reading the [Brightway2 manual](https://brightway2.readthedocs.org/en/latest/).

At the end of this tutorial, you will be able to:

* Import basic data like the biosphere database
* Import and explore the [FORWAST](http://forwast.brgm.fr/) database, which is used in the other tutorials.

If you finish the tutorial, you get a kitten.

This tutorial is written in an Jupyter notebook, an online scientific notebook which combines, text, data, images, and programming. It is amazing, and could be a fantastic way to do and communicate advanced LCA work. See the [documentation](http://ipython.org/ipython-doc/dev/interactive/htmlnotebook.html) and a list of [awesome examples](https://github.com/ipython/ipython/wiki/A-gallery-of-interesting-IPython-Notebooks).

You should **download this notebook** and run it cell by cell - don't just read it on the web!

## Brightway 2 tutorials

Please see the documentation for a complete list of tutorials and other example notebooks.

# Starting at the beginning

Import brightway2.

In [1]:
from brightway2 import *

## Python 2 and 3

Brightway2 is compatible with both Python 2 and 3. If you are using Python 2, I strongly recommend you execute the following cell, which will make all your text string unicode without have to type the letter ``"u"`` in front of each text string. On Python 3, the following **doesn't do anything** - your text strings are unicode by default.

In [2]:
from __future__ import unicode_literals, print_function

## Projects

In Brightway2, a project is a separate directory with its own copies of LCI databases, LCIA methods, and any other data you use. Each research project or article should probably be its own project, so that any changes you want to make will not interfere with your other work.

The default project is called ``default``:

In [3]:
projects.current

'default'

Each project is stored in a separate directory in a place in your filesystem reserved for application data. It varies depending on the operating system; on OS X, this is the `Library` directory in your user home directory:

In [4]:
projects.dir

'/Users/cmutel/Library/Application Support/Brightway3/default.c21f969b5f03d33d43e04f8f136e7682'

However, you really need to care about the specifics. 

We can create a new project:

In [5]:
projects.current = "BW2 Tutorial 1"

And list the available projects:

In [ ]:
list(projects)

## Getting basic data

Let's import some basic data - a database of elementary flows, some LCIA methods, and some metadata used for importing other databases:

In [7]:
bw2setup()

Creating default biosphere

Applying strategy: normalize_units

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 0.324 sec



Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.01 seconds
Title: Writing activities to SQLite3 database:
  Started: 11/06/2015 15:41:24
  Finished: 11/06/2015 15:41:24
  Total time elapsed: 0.324 sec
  CPU %: 96.100000
  Memory %: 0.331557
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 0.96 seconds
Wrote 692 LCIA methods with 170915 characterization factors
Creating core data migrations



The iPython notebook by default prints all logged messages. On your machine, there might be messages tracking how long it took to download and process the biosphere and methods packages.

## A biosphere dataset

The ``biosphere3`` database was installed. It is called the ``biosphere3`` database because elementary flow names are normalized to the ecoinvent 3 standard.

Let's see how many flows there are in this database, and then look at one random flow:

In [8]:
db = Database("biosphere3")
print("Number of flows in `biosphere3`:", len(db))
random_flow = db.random()
print(random_flow)

Number of flows in `biosphere3`: 3955
'Butene' (kilogram, None, ('water', 'ground-'))


In [9]:
print(random_flow['name'])
print(random_flow['unit'])
print(random_flow['categories'])

Butene
kilogram
('water', 'ground-')


Brightway2 uses keys to identify datasets. Each dataset is identified by a combination of its database and some unique code. The code can be anything - a number, a UUID, or just a name. All of the following would be valid keys:

    ("biosphere", "f66d00944691d54d6b072310b6f9de37")
    ("my new database", "building my dream house")
    ("skynet", 14832)

In [10]:
random_flow.key

('biosphere3', '8b36dc51-b9d4-4141-96d9-9183f6198a20')

## An LCIA method dataset

We also installed a large number of LCIA methods:

In [11]:
len(methods)

692

Because LCIA methods have many different impact categories, they are identified not by a single label, but by a list of labels. Let's look at an example:

In [12]:
method_key = methods.random()
method_key

('EDIP w/o LT',
 'resource consumption w/o LT',
 'non-renewable resources, gold w/o LT')

In this case, the LCIA method has three levels of specificity, from the general name (first level) to the specific impact category (last level). There is nothing magic about three levels - you could have one, or one hundred - but Brightway2 expects that LCIA methods will be a list of text labels ``('like', 'this')``.

Note that method identifiers need to be a special kind of list that uses `()` instead of `[]`. These are called `tuples`. To create a tuple with only one element, you need to add a comma, to distinguish it from a set of parentheses:

In [13]:
print (1 + 2)           # This is not a tuple
print (1,), type((1,))  # This is a tuple with one element

3
1


(None, tuple)

We can load the method data, show a sample.

Method data has the format:

    biosphere flow, numeric value, location

Where:

* `biosphere flow` is a dataset from any database which is used as a biosphere flow.
* `numeric value` can be either a fixed number or an uncertainty distribution.
* `location` is optional; the default value is that this characterization factor is valid everywhere.

The method data format is pretty flexible, and the following are all acceptable:

    [('biosphere', 'CO2'), 1.0],                                             # Default location
    [('biosphere', 'CO2'), 1.0, 'Australia, mate!'],                         # Custom location
    [('biosphere', 'CO2'), 1.0, ('Population density', 'Raster cell 4,2')],  # Location inside a geocollection
    [('biosphere', 'CO2'), {'amount': 1.0, 'uncertainty type': 0}],          # Uncertain characterization factor

[Geocollections](http://brightway2-regional.readthedocs.org/#spatial-scales-geocollections) are needed for regionalized LCA.

If you are wondering why we need to identify biosphere flows like `('biosphere', '2fe885840cebfcc7d56b607b0acd9359')`, this is a good question! The short answer is that there is no single field that uniquely identifies biosphere flows or activities. The longer answer [is in the manual](http://brightway2.readthedocs.org/en/latest/lci.html#uniquely-identifying-datasets).

Brightway2 is designed to be flexible enough for many different problems. Therefore, there are no limits on what constitutes a biosphere flow. Rather, anything that is linked to in a biosphere exchange will be put in the biosphere matrix. We installed a database called `biosphere3`, but you can define new flows in a database alongside process datasets, or create your own biosphere database.

In [14]:
method_data = Method(method_key).load()
print("Number of CFs:", len(method_data))
method_data[:20]

Number of CFs: 9


[(('biosphere3', 'ff741136-d6ee-444a-a15b-3b308e376db8'), 1.0),
 (('biosphere3', '7cd1d217-70a7-4452-abc4-3b1100763d6d'), 1.0),
 (('biosphere3', 'a8896ed6-4c9d-4b06-a356-49d8cdd9e9d7'), 1.0),
 (('biosphere3', '2d65a3f7-2a10-4a10-ac9e-a0cc7cd57979'), 1.0),
 (('biosphere3', '95268685-7bea-4883-a412-119d7e88372c'), 1.0),
 (('biosphere3', '8c888d2b-d608-4dac-bad5-1c2a17050838'), 1.0),
 (('biosphere3', '3eece329-cf79-4167-93c2-b8d7d7eb5058'), 1.0),
 (('biosphere3', 'd28f9d42-5df5-41c3-be59-fdfa7ff57112'), 1.0),
 (('biosphere3', 'cf3d3dbc-0e4b-402d-92a1-8ea6b4869ed5'), 1.0)]

# Importing the FORWAST LCI database

We will use the FORWAST database in these tutorials, as it is both a high quality, comprehensive LCI database, and freely available. [FORWAST](http://forwast.brgm.fr/Overview.asp) is a physical MRIO table for Europe. It can be downloaded directly from the [2.-0 website](http://lca-net.com/projects/show/forwast/).

The following cell will download and install the FORWAST database. Note that an internet connection is required.

In [15]:
import zipfile
import os
from bw2data.utils import download_file

filepath = download_file("forwast.bw2package.zip", url="http://lca-net.com/wp-content/uploads/")
dirpath = os.path.dirname(filepath)
zipfile.ZipFile(filepath).extractall(dirpath)
BW2Package.import_file(os.path.join(dirpath, "forwast.bw2package"))

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 2.241 sec


Title: Writing activities to SQLite3 database:
  Started: 11/06/2015 15:42:11
  Finished: 11/06/2015 15:42:13
  Total time elapsed: 2.241 sec
  CPU %: 92.200000
  Memory %: 0.841737


[Brightway2 SQLiteBackend: forwast]

That't it! The next tutorial will show you how to work with data in databases.

Here is your promised kitten:

In [16]:
from IPython.display import Image
import random
dimensions = sorted((int(random.random() * 600 + 200), int(random.random() * 600 + 200)))
Image(url="http://placekitten.com/{}/{}/".format(*dimensions))